# Querying the Knowledge Graph with Neo4j Cypher

Prereqs: 
    
*  The default Movies Neo4j database: Either [Neo4j Desktop](https://neo4j.com/download/) (local instance) or [Aura](https://neo4j.com/product/auradb/) (cloud instance)

In [1]:
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase

from importlib import reload
import utils

import warnings
warnings.filterwarnings('ignore')
#reload(utils) - this i needed if I do changes to utils.py


## What is Cypher?

Neo4j Cypher is a query language for the Neo4j graph database. It allows you to create, read, update, and delete nodes and relationships in a graph. Cypher is similar to SQL but designed for graph databases, making it easier to work with graph data. It's designed to express graph patterns.

### Connect to the Neo4j database 
Connect to the graph database using the [LangChain Neo4jGraph connector](https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html).

In [2]:
graph = Neo4jGraph()

### Query the graph database

Match all nodes in graph

In [3]:
cypher = """Match (n) return count(n)"""

result = graph.query(cypher)
result

[{'count(n)': 171}]

Match only the `Movie` nodes using the node label

In [4]:
cypher = """
MATCH (m:Movie) 
RETURN count(m) AS numberOfMovies
"""

result = graph.query(cypher)
result

[{'numberOfMovies': 39}]

Match only the `Person` nodes using the node label

In [5]:
cypher = """
MATCH (people:Person) 
RETURN count(people) AS numberOfPeople
"""

result = graph.query(cypher)
result

[{'numberOfPeople': 132}]

Match a single `Person` node using `name` property

In [6]:
cypher = """
MATCH (tom:Person {name: 'Tom Hanks'})
RETURN tom
"""

result = graph.query(cypher)
result

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

Match a single `Movie` node using `title` property

In [7]:
cypher = """
MATCH (apollo13:Movie {title: 'Apollo 13'})
RETURN apollo13
"""

result = graph.query(cypher)
result

[{'apollo13': {'tagline': 'Houston, we have a problem.',
   'title': 'Apollo 13',
   'released': 1995}}]

Match movies released in the 1990s and return their titles.

In [8]:
cypher = """
MATCH (nineties:Movie) 
WHERE nineties.released >= 1990 AND nineties.released < 2000 
RETURN nineties.title
"""

result = graph.query(cypher)
result

[{'nineties.title': 'The Matrix'},
 {'nineties.title': "The Devil's Advocate"},
 {'nineties.title': 'A Few Good Men'},
 {'nineties.title': 'As Good as It Gets'},
 {'nineties.title': 'What Dreams May Come'},
 {'nineties.title': 'Snow Falling on Cedars'},
 {'nineties.title': "You've Got Mail"},
 {'nineties.title': 'Sleepless in Seattle'},
 {'nineties.title': 'Joe Versus the Volcano'},
 {'nineties.title': 'When Harry Met Sally'},
 {'nineties.title': 'That Thing You Do'},
 {'nineties.title': 'The Birdcage'},
 {'nineties.title': 'Unforgiven'},
 {'nineties.title': 'Johnny Mnemonic'},
 {'nineties.title': 'The Green Mile'},
 {'nineties.title': 'Hoffa'},
 {'nineties.title': 'Apollo 13'},
 {'nineties.title': 'Twister'},
 {'nineties.title': 'Bicentennial Man'},
 {'nineties.title': 'A League of Their Own'}]

Match all `Person`s who have co-acted with Tom Hanks in any `Movie`

In [9]:
cypher = """
MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(p:Person) 
RETURN p.name, m.title
"""

result = graph.query(cypher)
result

[{'p.name': 'Meg Ryan', 'm.title': "You've Got Mail"},
 {'p.name': 'Greg Kinnear', 'm.title': "You've Got Mail"},
 {'p.name': 'Parker Posey', 'm.title': "You've Got Mail"},
 {'p.name': 'Dave Chappelle', 'm.title': "You've Got Mail"},
 {'p.name': 'Steve Zahn', 'm.title': "You've Got Mail"},
 {'p.name': 'Meg Ryan', 'm.title': 'Sleepless in Seattle'},
 {'p.name': 'Rita Wilson', 'm.title': 'Sleepless in Seattle'},
 {'p.name': 'Bill Pullman', 'm.title': 'Sleepless in Seattle'},
 {'p.name': 'Victor Garber', 'm.title': 'Sleepless in Seattle'},
 {'p.name': "Rosie O'Donnell", 'm.title': 'Sleepless in Seattle'},
 {'p.name': 'Meg Ryan', 'm.title': 'Joe Versus the Volcano'},
 {'p.name': 'Nathan Lane', 'm.title': 'Joe Versus the Volcano'},
 {'p.name': 'Charlize Theron', 'm.title': 'That Thing You Do'},
 {'p.name': 'Liv Tyler', 'm.title': 'That Thing You Do'},
 {'p.name': 'Hugo Weaving', 'm.title': 'Cloud Atlas'},
 {'p.name': 'Halle Berry', 'm.title': 'Cloud Atlas'},
 {'p.name': 'Jim Broadbent', 'm.

Match `Movie`s and `Actor`s that are 3 hops away from Kevin Bacon. (From [Neo4j graph examples - movies](https://github.com/neo4j-graph-examples/movies))

In [10]:
cypher = """
MATCH (p:Person {name: 'Kevin Bacon'})-[*1..3]-(hollywood) 
RETURN DISTINCT p, hollywood
"""

result = graph.query(cypher)
result


[{'p': {'born': 1958, 'name': 'Kevin Bacon'},
  'hollywood': {'tagline': "In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.",
   'title': 'A Few Good Men',
   'released': 1992}},
 {'p': {'born': 1958, 'name': 'Kevin Bacon'},
  'hollywood': {'tagline': '400 million people were waiting for the truth.',
   'title': 'Frost/Nixon',
   'released': 2008}},
 {'p': {'born': 1958, 'name': 'Kevin Bacon'},
  'hollywood': {'tagline': 'Houston, we have a problem.',
   'title': 'Apollo 13',
   'released': 1995}},
 {'p': {'born': 1958, 'name': 'Kevin Bacon'},
  'hollywood': {'born': 1962, 'name': 'Tom Cruise'}},
 {'p': {'born': 1958, 'name': 'Kevin Bacon'},
  'hollywood': {'born': 1937, 'name': 'Jack Nicholson'}},
 {'p': {'born': 1958, 'name': 'Kevin Bacon'},
  'hollywood': {'born': 1962, 'name': 'Demi Moore'}},
 {'p': {'born': 1958, 'name': 'Kevin Bacon'},
  'hollywood': {'born'